### Read the script

In [ ]:
import os
from ieeg.io import get_data, raw_from_layout, save_derivative
from ieeg.navigate import crop_empty_data

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = subjects[1]

In [ ]:
raw = raw_from_layout(layout.derivatives['derivatives/clean'], subject=subject, preload=False, desc='clean',extension='.edf')
# Choose preload=False to release RAM space (but )
raw = crop_empty_data(raw)

### Remove bad channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html


In [ ]:
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad
from ieeg.navigate import channel_outlier_marker, crop_empty_data
from ieeg.viz.parula import parula_map
import numpy as np
import mne

In [ ]:
# plot subject
# !!!!! problems: cannot find the surface file##########
from ieeg.viz.mri import plot_subj, plot_on_average
# plot the subject brain
fig1 = plot_subj('D103')
#C:\Users\bl314\Box\ECoG_Recon\D103
#Label every electrode
mne.viz.set_3d_view(fig1, azimuth=150, elevation=70, focalpoint="auto",
                    distance="auto")

In [ ]:
# Remove EEG channels for D101
# eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']
# Remove EEG channels for D103 (same)
eeg_channels_to_exclude = ['F7', 'F8', 'T5', 'T6', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4',
                           'T3', 'T4', 'O1', 'O2']
# Problems: !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!Channel(s) Fz, Cz, Pz, Fp1, Fp2 not found, nothing dropped. But these channels exist in the sub-D0103_task-LexicalDecRepDelay_channels.tsv!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Remove EEG channels for D107

raw.drop_channels(eeg_channels_to_exclude)
#raw.load_data()

In [ ]:
# mark channel outliers as bad
raw.info['bads'] = channel_outlier_marker(raw, 3, 2)
# Exclude bad channels
raw.drop_channels(raw.info['bads'])

In [ ]:
# CAR (common average reference) DONT DO THIS NOW
# raw.set_eeg_reference(ref_channels="average", ch_type='seeg')

In [ ]:
raw.plot()

### Remove bad trials and plot spectrogram
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [ ]:
from ieeg.navigate import channel_outlier_marker, trial_ieeg,outliers_to_nan

In [ ]:
# Epoches: https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.__getitem__

epoch = 'Auditory_stim'
#epoch = 'Delay'
times = (-1, 2)
trials = trial_ieeg(raw, epoch, times, preload=True)
# eeg file is big. If try load multiple subjects. 
# pointing
outliers_to_nan(trials, outliers=10)

In [ ]:
# baseline = 'Cue'
# times = [-0.6, 0]
# baselines = trial_ieeg(raw, baseline, times, preload=True)
# outliers_to_nan(baselines, outliers=10)

### Multitaper spectrogram plot

#### Calculate Spectra

In [ ]:
# Multitaper wavelet timefreq
#freq = np.arange(10, 200., 6.)
#spectra = spectrogram(raw, freq, 'Auditory_stim', -1.2, 1.2, 'Cue', -0.5, 0,
#                      n_jobs=-3, verbose=10, time_bandwidth=10, n_cycles=freq/2)
# verbose: extra messages
# https://mne.tools/dev/generated/mne.verbose.html#mne.verbose
#spectra = spectrogram(trials, baseline=baselines, freqs=freq, n_jobs=-3, time_bandwidth=10, n_cycles=freq/2)
# https://mne.tools/dev/generated/mne.time_frequency.tfr_array_multitaper.html (Notes)
#crop_pad(spectra, "0.5s")

In [ ]:
#Learned from Aaron's sentence rep
#https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
freq = np.linspace(0.5, 200, num=80)
kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
              n_cycles=freq/2, time_bandwidth=4,
              # n_fft=int(trials.info['sfreq'] * 2.75),
              decim=20, )
              # adaptive=True)
spectra = trials.compute_tfr(method="multitaper",  **kwargs)
crop_pad(spectra, "0.5s") # cut the first and final 0.5s, change to zero

In [ ]:
base = spectra.copy().crop(-0.5, 0)
# base = baselines.compute_tfr(method="multitaper",  **kwargs)
# crop_pad(base, "0.5s")

In [ ]:
from ieeg.calc.scaling import rescale
# del trials, baselines
base = base.average(lambda x: np.nanmean(x, axis=0), copy=True)
spectra = spectra.average(lambda x: np.nanmean(x, axis=0), copy=True)
rescale(spectra._data, base._data, mode='ratio', axis=-1)

In [ ]:
#fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
#spectra.info['subject_info']['files'] = tuple(fnames)
spectra.info['bads'] = raw.info['bads']
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'
#https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
if not os.path.exists(os.path.join(save_dir, subject)):
    os.mkdir(os.path.join(save_dir, subject))
filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
#spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))


#### Plot data

In [ ]:
del spectra
filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
spectra = mne.time_frequency.read_tfrs(filename)


In [ ]:
chan_grid(spectra, size = (20,10),vlim=(0.7, 1.4), fmax=200, show=True, cmap=parula_map,)# yscale="log")
# MNE bsl correction does not allow